In [ ]:
!pip install -e /Users/esummer/workspace/python/my-python-utils-main

In [ ]:
import json
import os

from slugify import slugify
import csv

import logging
import time
import random
import pandas as pd
from datetime import datetime
from zoneinfo import ZoneInfo

import requests
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile

from zoneinfo import ZoneInfo

import re

In [ ]:
my_items = '''MMKGSVE04BK001
MMKGSVE05BK001
MMKGSVE06BK001
MMKGSVE08CO001
MMKGSVE09BK001
MMLGSTA02BK001
MMLGSTA03BK001
MMRGSAK01BK001
MMRGSAK01CO001
MMRGSAK02BK001
MMRGSAK02CO001
MMRGSTA05BK001
MMRGSTA06SV001
MMRGSTA07PZ001
MMTGSTA01CO001
MMTGSTA02CO001
MMTGSTA03BK001
MMTGSTA04BK001
MMTGSTA04CO001
MMTGSTA05CO001
MMZGSFI01BK001
MMZGSFI02BK001
MMZGSFI03CO001
MWBGSEA01BK001
MWBGSEA03Q6001
MWBGSEA03QA001
MWBGSEA04CO001
MWBGSEA05BK001
MWBGSEA06BK001
MWBGSEA07BK001
MWCGSAK01BK001
MWDGSDU01BK001
MWDGSDU01Q6001
MWDGSDU01QA001
MWDGSDU02CO001
MWDGSDU03CO001
MWEGSXT01CO001
MWHGSTA03BK001
MWHGSTA03Q6001
MWHGSTA03QA001
MWHGSTA04BK001
MWHGSTA04PZ001
MWKGSAK01BK001
MWKGSTA01Q6001
MWKGSTA01QA001
MWPGSAC01L8001
MWPGSLR02BK001
MWPGSLR02CO001
MWPGSLR02PZ001
MWPGSLR03BK001
MWPGSLR03CO001
MWPGSLR03PZ001
MWPGSLR04BK001
MWPGSLR04CO001
MWPGSLR05BK001
MWPGSLR05CO001
MWPGSMT04BK001
MWPGSMT04QA001
MWPGSMT05BK001
MWPGSMT06CO001
MWPGSMT07CO001
MWPGSMT10BK001
MWPGSMT12L8001
MWPGSTA01PZ001
MWRGSTA01BK001
MWRGSTA01OQ001
MWRGSTA02CO001
MWRGSTA02PZ001
MWRGSTA03BK001
MWRGSXT01BK001
MWRGSXT01CO001
MWSGSAK01BK001
MWSGSTA01BK001
MWSGSTA01OQ001
MWSGSTA02CO001
MWSGSTA04BW001
MWSGSTA05BW001
MWSGSTA05BW001
MWTGSBO02BK001
MWTGSMA01BK001
MWTGSMA02BK001
MWTGSMA02QA001
MWTGSTA02BK001
MWTGSTA03PZ001
MWZGSAK01BK001
MXBGSCJ01BK001
MXBGSCJ02CO001
MXBGSTA02BK080
MXBGSTA03BK080
MXBGSTA04BK001
MXBGSTA05BK001
MXZFAAK01BK001
MXZGSAK02BK001
MXZGSKS01BK001
MYZDSTA09PZ001
MYZFSTA01Q6001
MYZGSTA01CO001
MYZGSTA02CO001
MYZGSTA03BK001
MYZGSTA04L8001
MYZGSTA05CO001
MYZGSTA06BK001
MYZGSTA06CO001
MYZGSTA07QA001
MYZGSTA08PZ001'''.split('\n')



import random


# 임의로 10개 선택
selected_items = random.sample(my_items, 10)

print(selected_items)

df = pd.DataFrame(selected_items, columns=['SKU'])

## 기준정보 테이블 읽기

In [ ]:
df_products = pd.read_excel('../excel-input/00.기준정보 테이블__Products.xlsx', header=0)
df_regions = pd.read_excel('../excel-input/00.기준정보 테이블__Regions.xlsx', header=0)
df_vendor = pd.read_excel('../excel-input/00.기준정보 테이블__Vendors.xlsx', header=0)
df_catetorys = pd.read_excel('../excel-input/00.기준정보 테이블__Catetorys.xlsx', header=0)

## Production 마스터 데이터 읽어 오기

In [59]:
df_production_origin = pd.read_excel('../excel-input/00.3.Production 마스터 (생산본부)__Production Sample.xlsx', header=0)
df_production_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10786 entries, 0 to 10785
Data columns (total 60 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   NO                                  10786 non-null  int64         
 1   LAUNCHING                           3613 non-null   object        
 2   PO DATE                             10786 non-null  datetime64[ns]
 3   Program                             10786 non-null  object        
 4   Order Type                          10786 non-null  object        
 5   Category                            10786 non-null  object        
 6   Vendor                              10786 non-null  object        
 7   Channel                             10786 non-null  object        
 8   PO No                               10786 non-null  object        
 9   PO No (SAP)                         10771 non-null  object        
 10  (Contractual) Ex Fty M

In [62]:
filtered_notna = df_production_origin[df_production_origin['PO  Q\'ty'].notna()]

# filtered_notna[:10].to_json('../excel-output/df_production_sample_template#1.json', orient='records')
filtered_notna[:10][['item',"PO  Q'ty", '(Contractual) Ex Fty Month','Category','Line Name', 'Order Type','Program','(GMD) Contractual Ex fty','Vendor', 'Channel']].to_json('../excel-output/df_production_sample_template#1_filter.json', orient='records')


## 주문 데이터의 각 컬럼별 info()

In [ ]:
from io import StringIO

df_production_sample = df_production_origin.copy()

df_production_sample.columns = df_production_sample.columns.str.replace(' ', '__')


buf = StringIO()
df_production_sample.info(buf=buf)
info_str = buf.getvalue()

# 2. info 문자열을 줄 단위로 나눔
lines = info_str.strip().split("\n")

# 3. 컬럼 정보 라인만 추출 (인덱스, 컬럼명, Non-Null Count, Dtype)
data_lines = [line.strip() for line in lines if "non-null" in line]

# 4. 공백 기준 분리 후 DataFrame 생성
info_data = []
for line in data_lines:
    parts = line.split()
    col_name = parts[1]
    non_null = parts[2]
    dtype = parts[-1]
    info_data.append([col_name, non_null, dtype])

info_df = pd.DataFrame(info_data, columns=["Column", "Non-Null Count", "Dtype"])

# 5. 엑셀로 저장
info_df.to_excel("../excel-output/df_production_sample_info.xlsx", index=False)
df_production_sample.to_excel("../excel-output/df_production_sample_data.xlsx", index=False)

## 코드성 컬럼값 추출

In [ ]:
# a 컬럼의 값 종류별 카운트
columns = ['Order__Type', 'Program','Category',	'Vendor',	'Channel',	'Line__Name', 'PO__No__(SAP)', 'PO__No']
for column in columns:
    count_df = df_production_sample[column].value_counts().reset_index()

    # 컬럼명 변경
    count_df.columns = [f"{column} 값", "count"]
    count_df.to_excel(f"../excel-output/df_production_컬럼값_{column}_data-{len(count_df)}-건.xlsx", index=False)

In [60]:
df_production_sample[:10].to_json('../excel-output/df_production_sample_template_#2.json', orient='records')

In [61]:
df_production_sample_2 = pd.read_excel('../excel-input/★1. Q12 26(SS26) Selling_initial_out_0630_V2_Sample.xlsx')
df_production_sample_2.info()

filtered_notna = df_production_sample_2[df_production_sample_2['WW ORDER QTY'].notna()]

# filtered_notna[:10].to_json('../excel-output/df_production_sample_2_all.json', orient='records')
filtered_notna[:10][['ITEM#','WW ORDER QTY','MOQ','EX-FTY DATE','CATEGORY','LINE NAME','GMD MASTER ORDER','LAUNCHING MONTH','DEPARTMENT','OTB Cate']].to_json('../excel-output/df_production_sample_template#2_filter.json', orient='records')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519 entries, 0 to 518
Columns: 392 entries, No to GB
dtypes: float64(9), int64(1), object(382)
memory usage: 1.6+ MB


## 코드성 컬럼의 숫자 집계

In [ ]:
unique_values = df_production_sample[['Order__Type', 'Program']].drop_duplicates()
unique_values.sort_values(by=['Order__Type', 'Program'], ascending=[True,True]).to_excel(f"../excel-output/df_production_컬럼값_[Order__Type & Program]_data-{len(unique_values)}-건.xlsx", index=False)

unique_values = df_production_sample[['Order__Type', 'Category']].drop_duplicates()
unique_values.sort_values(by=['Order__Type', 'Category'], ascending=[True,True]).to_excel(f"../excel-output/df_production_컬럼값_[Order__Type & Category]_data-{len(unique_values)}-건.xlsx", index=False)

unique_values = df_production_sample[['Category', 'Order__Type']].drop_duplicates()
unique_values.sort_values(by=['Category', 'Order__Type'], ascending=[True,True]).to_excel(f"../excel-output/df_production_컬럼값_[Category & Order__Type]_data-{len(unique_values)}-건.xlsx", index=False)

unique_values = df_production_sample[['Program', 'Order__Type']].drop_duplicates()
unique_values.sort_values(by=['Program', 'Order__Type'], ascending=[True,True]).to_excel(f"../excel-output/df_production_컬럼값_[Program & Order__Type]_data-{len(unique_values)}-건.xlsx", index=False)


## 코드성 컬럼의 그룹바이 집계

In [ ]:
counts_groupby = df_production_sample.groupby(['Category', 'Order__Type']).size().reset_index(name='건수')
counts_groupby.to_excel(f"../excel-output/df_production_컬럼값_[Category & Order__Type]_groupby-{len(counts_groupby)}-건.xlsx", index=True)

In [ ]:
counts_groupby = df_production_sample.groupby(['Line__Name', 'item']).size().reset_index(name='건수')
counts_groupby.to_excel(f"../excel-output/df_production_컬럼값_[Line__Name & item]_groupby-{len(counts_groupby)}-건.xlsx", index=True)

counts_groupby = df_production_sample.groupby(['item', 'Line__Name']).size().reset_index(name='건수')
counts_groupby.to_excel(f"../excel-output/df_production_컬럼값_[item & Line__Name]_groupby-{len(counts_groupby)}-건.xlsx", index=True)

## Production Head 구분 기준

In [ ]:
counts_groupby = df_production_sample.groupby(['Program', 'Order__Type', 'Channel', '(Contractual)__Ex__Fty__Month']).size().reset_index(name='건수')
counts_groupby.to_excel(f"../excel-output/df_production_컬럼값_[Program & Order__Type & Channel]_groupby-{len(counts_groupby)}-건.xlsx", index=True)

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
#
# html = df_production_sample.to_html(index=False)
# Path("df_simple.html").write_text(html, encoding="utf-8")

from pydantic_settings import BaseSettings

import pandas as pd
from pandas_profiling import ProfileReport

profile = ProfileReport(df_production_sample)

profile.to_file(output_file=f"df_production_sample.html") # html


In [ ]:
# !pip uninstall pandas-profiling
!pip install pandas-profiling

In [ ]:
!python3 -m pip install --upgrade pip

In [ ]:
 !pip install pydantic-settings